In [1]:
import torch
import numpy as np
from torch_sparse.tensor import SparseTensor
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

from sketch import CountSketch, TensorSketch
from sketch import Backends, OutputTypes, SketchConfig, DEFAULT_COUNT_SKETCH_CONFIG, DEFAULT_TENSOR_SKETCH_CONFIG
from initialize_sketch import initialize_single_layer_sketch_modules, preprocess_data

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor(), root="../dataset")
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()

select_ratio = 1/16
compress_ratio = 0.4
n_layers = 3
order = 3

in_dim = int(data.x.size(0) * select_ratio)
out_dim = int(in_dim * compress_ratio)
conv_mat = data.adj_t[:in_dim,:in_dim]
node_feature_mat = data.x[:in_dim,:]
node_labels = data.y[:in_dim]

print("Time on GPU: ", end="")
%timeit conv_sketches, nf_sketches, ll_cs_list = preprocess_data(n_layers, in_dim, out_dim, order, conv_mat, node_feature_mat, device=torch.device('cuda:0'))
print("Time on CPU: ", end="")
%timeit conv_sketches, nf_sketches, ll_cs_list = preprocess_data(n_layers, in_dim, out_dim, order, conv_mat, node_feature_mat, device=torch.device('cpu'))

Time on GPU: 

RuntimeError: The size of tensor a (128) must match the size of tensor b (10583) at non-singleton dimension 0